In [181]:
import pandas as pd
import numpy as np
import os

In [182]:
basepath=("./datasets/")
pandas_list=[]

for item in os.listdir(basepath):
    print(item)
    fn=(os.path.join(basepath, item))  #create a file handler 
    obj=pd.read_excel(fn,encoding="utf-8")                #open file as a pandas dataframe  
    pandas_list.append(obj)

UFC_Social.xlsx
UFC_PPV_Data.xlsx


In [183]:
pandas_list[0].head()

,Athlete,Category,Gender,Nation,Rank,Active,Twitter,Instagram,Facebook
0,Daniel Cormier,Heavyweight,M,USA,C,YES,709000,1900000,NaN
1,Stipe Miocic,Heavyweight,M,USA,1,YES,239000,774000,NaN
2,Francis Ngannou,Heavyweight,M,FRA,2,YES,99700,497000,NaN
3,Derrick Lewis,Heavyweight,M,USA,3,YES,148000,1500000,NaN
4,Curtis Blaydes,Heavyweight,M,USA,4,YES,12700,79100,NaN


In [184]:
pandas_list[1].head()

,Event,Date,Venue,City,State/Province,Country,Attendance,EVENT #,PPV,Explanation,Month,Year,Extra Notes
0,UFC 1: The Beginning,1993-11-12,McNichols Sports Arena,Denver,Colorado,U.S.,7800,UFC 1,Not Available,Pre-PPV,November,1993,NaN
1,UFC 2: No Way Out,1994-03-11,Mammoth Gardens,Denver,Colorado,U.S.,2000,UFC 2,Not Available,Pre-PPV,March,1994,NaN
2,UFC 3: The American Dream,1994-09-09,Grady Cole Center,Charlotte,North Carolina,U.S.,NaN,UFC 3,Not Available,Pre-PPV,September,1994,NaN
3,UFC 4: Revenge of the Warriors,1994-12-16,Expo Square Pavilion,Tulsa,Oklahoma,U.S.,5857,UFC 4,Not Available,Pre-PPV,December,1994,NaN
4,UFC 5: The Return of the Beast,1995-04-07,Independence Arena,Charlotte,North Carolina,U.S.,6000,UFC 5,Not Available,Pre-PPV,April,1995,NaN


In [185]:
social_stats=pandas_list[0]
ppv_data=pandas_list[1]

### Cleanup social_stats dataset

**Twitter | Instagram** Columns are formated as strings. I will convert them into integrers and convert **`NaN`** values
into **0**.

Refs:
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html

In [186]:
columns=['Twitter','Instagram']

for c in columns:
    social_stats[c].fillna(value=0,inplace=True)
    social_stats[c]=social_stats[c].astype(dtype='int64',errors='ignore',)

In [187]:
social_stats['Twitter'].head()

0    709000
1    239000
2     99700
3    148000
4     12700
Name: Twitter, dtype: object

In [188]:
social_stats['Instagram'].head()

0    1900000
1     774000
2     497000
3    1500000
4      79100
Name: Instagram, dtype: object

### Check NaN in Facebook colum

It seems that **`Facebook`** column has a lot of NaN values, let's check that a little bit further. We might want to drop what column if there is no usefull information.

In [189]:
social_stats['Facebook'].value_counts()

15.12.2018    1
Name: Facebook, dtype: int64

* Only one value different then `NaN`
* Check the last 10 rows

In [190]:
social_stats.tail(10)

,Athlete,Category,Gender,Nation,Rank,Active,Twitter,Instagram,Facebook
161,Wanderlei Silva,Light Heavyweight,M,BRA,-,NO,794000,600000,NaN
162,Cain Velasquez,Heavyweight,M,BRA,-,YES,894000,954000,NaN
163,Fabricio Werdum,Heavyweight,M,BRA,-,YES,341000,833000,NaN
164,Mirko Filipovic,Heavyweight,M,CRO,-,YES,-,94700,NaN
165,Alexander Gustafsson,Light Heavyweight,M,SWE,-,YES,273000,616000,NaN
166,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
167,AVERAGE,NaN,NaN,NaN,NaN,NaN,288324,710028,NaN
168,MEDIAN,NaN,NaN,NaN,NaN,NaN,51100,126500,NaN
169,MAXIMUM,NaN,NaN,NaN,NaN,NaN,7860000,29700000,NaN
170,NaN,NaN,NaN,NaN,NaN,NaN,0,data updated:,15.12.2018


In [193]:
social_stats.drop('Facebook',axis=1, inplace=True)
social_stats.head()

,Athlete,Category,Gender,Nation,Rank,Active,Twitter,Instagram
0,Daniel Cormier,Heavyweight,M,USA,C,YES,709000,1900000
1,Stipe Miocic,Heavyweight,M,USA,1,YES,239000,774000
2,Francis Ngannou,Heavyweight,M,FRA,2,YES,99700,497000
3,Derrick Lewis,Heavyweight,M,USA,3,YES,148000,1500000
4,Curtis Blaydes,Heavyweight,M,USA,4,YES,12700,79100
